# Kickstarter and SQL `10 points`

Since I claimed that the point of Lede was to be able to learn anything using Google, I should probably test that, too, yeah?

Working with **SQL databases** is something we didn't learn! It comes in a lot of formats - PostreSQL, MySQL, Sqlite - but they are all generally the same thing.

SQL is great for when you're reading in big big big datasets: instead of waiting an hour for `.read_csv` to crash on a giant file that you're going to filter anyway, instead you just ask SQL to give you a subset of the data to work on.

**Topics**

* Learning new things with the power of the internet
* SQL

### The data

**Kickstarter** is a website people use to raise money for projects. It used to be really popular, but it's since died down a bit. We're going to work with [a dataset about Kickstarter in SQLite format](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/EOYBXM).

You'll want to download the database by going to **Access Dataset** and then **Download ZIP**.

It's a big file, and it's compressed! We'll be using the `kickstarter.db` file that is inside the zip file: when you open up the zip, I extracted it using [The Unarchiver](https://theunarchiver.com/) by telling it to open up the `.z01` file (it's so big it's spread across both the `.zip` and the `.z01`). 

## Open the dataset

Just like an Excel file can have different sheets, a SQL database has different tables.

In [31]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', '{:,.2f}'.format)

SQLite + pandas doesn't use the "normal" way to ask for tables, so I'll give that line to you.

In [2]:
import sqlite3

# Open a connection
conn = sqlite3.connect("kickstarter.db")

# Make a query, save it into a dataframe
df = pd.read_sql_query('SELECT name FROM sqlite_master WHERE type = "table";', conn)

# Close the connection
conn.close()

# See the results in the dataframe
df

,name
0,urls_to_scrape
1,all_files
2,category
3,creator
4,funding_trend
5,item
6,livestream
7,location
8,project
9,reward


## Simple selects `3 points`

### Select all of the data for the first 4 kickstarter projects

They live in the `project` table. This will help you understand how SQL queries work.

In [21]:
conn = sqlite3.connect("kickstarter.db")
df1 = pd.read_sql_query("SELECT * from project LIMIT 4", conn)
conn.close()
df1

,id,state,url_project,url_project_short,name,country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
0,22807353,canceled,https://www.kickstarter.com/projects/mikewood/...,http://kck.st/1UNDF6C,"ViperSharp, The Best Precision Knife Sharpener...",US,1750652465,23418188,28,1453170931,1462402800,1472168774,1461010001,NaN,1458918033,10000,"31,131.00",USD,$,"31,131.00",1,255,53,22,False,False,The ViperSharp offers infinite angles for shar...,True,False,https://ksr-ugc.imgix.net/assets/012/357/781/2...,None,0.80,None,2328882,None,None,None,None,None,2328882,True,False,inactive,1453170931,None,vipersharp-the-best-precision-knife-sharpener,https://www.kickstarter.com/projects/mikewood/...,https://www.kickstarter.com/projects/mikewood/...,"657,517.00",https://ksr-video.imgix.net/projects/2289308/v...,https://ksr-video.imgix.net/projects/2289308/v...,360.00,640.00,successful,www.kickstarter.com/projects/mikewood/vipersha...,2018-05-24 02:33:34,0.00
1,22810934,successful,https://www.kickstarter.com/projects/cavinboun...,http://kck.st/17xiZX3,"Lights, Coma, Action!",US,1157451606,2357536,48,1380594589,1383413613,1488327585,1383413613,"1,383,413,613.00",1380821613,15000,"18,167.88",USD,$,"18,167.88",1,284,4,40,True,False,A book to tell the story of who I was (LIGHTS)...,True,False,https://ksr-ugc.imgix.net/assets/011/584/530/2...,None,0.80,A book to tell the story of how I sustained a ...,716409,None,Adventures in Brain Injury,None,http://www.adventuresinbraininjury.com,"Lights, Coma, Action!",716409,True,False,active,1432157366,None,lights-coma-action,https://www.kickstarter.com/projects/cavinboun...,https://www.kickstarter.com/projects/cavinboun...,"297,295.00",https://ksr-video.imgix.net/projects/700251/vi...,https://ksr-video.imgix.net/projects/700251/vi...,360.00,640.00,successful,www.kickstarter.com/projects/cavinbounce/light...,2018-05-24 02:26:07,0.00
2,22821161,failed,https://www.kickstarter.com/projects/684178251...,https://www.kickstarter.com/projects/684178251...,Sentio Golf Putters: Feel IS the difference,US,684178251,2351810,28,1433725049,1446429290,1463739059,1446429290,NaN,1442537690,50000,"9,707.00",USD,$,"9,707.00",1,41,0,4,False,False,Choose the feel YOU want with our patented flo...,True,False,https://ksr-ugc.imgix.net/assets/012/161/151/b...,None,0.80,None,1945377,None,None,None,None,None,1945377,True,False,inactive,1433725049,None,sentio-golf-putters-feel-is-the-difference,https://www.kickstarter.com/projects/684178251...,https://www.kickstarter.com/projects/684178251...,"579,025.00",https://ksr-video.imgix.net/projects/1905803/v...,https://ksr-video.imgix.net/projects/1905803/v...,480.00,640.00,successful,www.kickstarter.com/projects/684178251/sentio-...,2017-03-11 01:20:37,NaN
3,22823613,failed,https://www.kickstarter.com/projects/123754320...,http://kck.st/2kL47iy,Brainade,US,1237543205,2497409,342,1486073091,1489778686,1489778687,1489778687,NaN,1487190286,750,1.00,USD,$,1.00,1,1,0,0,False,False,We intend to create an online platform for ind...,True,False,https://ksr-ugc.imgix.net/assets/015/387/055/f...,None,0.80,None,2862898,None,None,None,None,None,2862898,True,False,inactive,1486073091,None,brainade,https://www.kickstarter.com/projects/123754320...,https://www.kickstarter.com/projects/1237543

### Select the top 10 projects with the highest amount pledged

I only want to see the name and the amount pledged. **The number one result should be the Pebble watch** - if your result is something Japanese, try querying all of the columns and figure out what went wrong.

In [30]:
conn = sqlite3.connect("kickstarter.db")
df2 = pd.read_sql_query('select name, usd_pledged from project order by usd_pledged desc limit 10', conn)
conn.close()
df2

,name,usd_pledged
0,"Pebble Time - Awesome Smartwatch, No Compromises","20,338,986.27"
1,COOLEST COOLER: 21st Century Cooler that's Act...,"13,285,226.36"
2,"Pebble 2, Time 2 + All-New Pebble Core","12,779,843.49"
3,Kingdom Death: Monster 1.5,"12,393,139.69"
4,Pebble: E-Paper Watch for iPhone and Android,"10,266,845.74"
5,The World's Best TRAVEL JACKET with 15 Feature...,"9,192,055.66"
6,Exploding Kittens,"8,782,571.99"
7,OUYA: A New Kind of Video Game Console,"8,596,474.58"
8,"THE 7th CONTINENT – What Goes Up, Must Come Down.","7,072,757.00"
9,"The Everyday Backpack, Tote, and Sling","6,565,782.50"


## Filtering your queries `3 points`

### Find me – yes, me, Soma! – in the project creators table

In [58]:
conn = sqlite3.connect("kickstarter.db")
df3 = pd.read_sql_query("select * from creator WHERE name like 'Jonathan Soma'", conn)
conn.close()
df3

,id,name,slug,avatar_thumb,avatar_small,urls_web_user,avatar_medium,is_registered,urls_api_user,chosen_currency
0,546599331,Jonathan Soma,soma,https://ksr-ugc.imgix.net/assets/005/801/114/d...,https://ksr-ugc.imgix.net/assets/005/801/114/d...,https://www.kickstarter.com/profile/soma,https://ksr-ugc.imgix.net/assets/005/801/114/d...,None,https://api.kickstarter.com/v1/users/546599331...,None


### Find all of my projects

In [57]:
conn = sqlite3.connect("kickstarter.db")
df4 = pd.read_sql_query("select * from project WHERE creator_id like '546599331'", conn)
conn.close()
df4

,id,state,url_project,url_project_short,name,country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
0,320516934,successful,https://www.kickstarter.com/projects/soma/broo...,http://kck.st/d9d9Sx,Brooklyn Brainery Needs a Home!,US,546599331,12589335,1,1270872639,1278129540,1397755136,1278129615,1278129615,1271182430,9500,"9,629.95",USD,$,"9,629.95",1,204,9,9,True,True,"We host awesomely cheap, collaborative classes...",True,False,https://ksr-ugc.imgix.net/assets/011/261/121/e...,None,0.80,None,4894,None,None,None,None,None,4894,True,False,inactive,1425915800,None,brooklyn-brainery-needs-a-home,https://www.kickstarter.com/projects/soma/broo...,https://www.kickstarter.com/projects/soma/broo...,"3,709.00",https://ksr-video.imgix.net/projects/4746/vide...,None,314.00,560.00,successful,www.kickstarter.com/projects/soma/brooklyn-bra...,2018-05-24 05:28:04,0
1,1052891075,successful,https://www.kickstarter.com/projects/soma/cook...,http://kck.st/aZ8hta,Cooking Class in a Box [Exotic Cuisine Edition],US,546599331,12589335,10,1276719823,1278991380,1397756801,1278991807,1278991807,1276792614,200,"1,820.44",USD,$,"1,820.44",1,53,8,7,True,True,I'll build you a cooking class on an exotic cu...,True,False,https://ksr-ugc.imgix.net/assets/011/263/591/4...,None,0.80,None,7857,None,None,None,None,None,7857,True,False,inactive,1425915800,None,cooking-class-in-a-box-exotic-cuisine-edition,https://www.kickstarter.com/projects/soma/cook...,https://www.kickstarter.com/projects/soma/cook...,NaN,None,None,NaN,NaN,None,www.kickstarter.com/projects/soma/cooking-clas...,2018-05-25 09:41:14,0
2,1606179513,successful,https://www.kickstarter.com/projects/soma/ice-...,http://kck.st/cgpgNy,Ice Cream Club,US,546599331,2459115,10,1271981825,1274055420,1397755379,1274056206,1274056206,1272034766,220,485.98,USD,$,485.98,1,27,7,5,True,True,When it's 100 degrees outside you'll wish you ...,True,False,https://ksr-ugc.imgix.net/assets/011/261/487/c...,None,0.80,None,5368,None,None,None,None,None,5368,True,False,inactive,1425915800,None,ice-cream-club,https://www.kickstarter.com/projects/soma/ice-...,https://www.kickstarter.com/projects/soma/ice-...,NaN,None,None,NaN,NaN,None,www.kickstarter.com/projects/soma/ice-cream-club,2018-05-27 00:19:54,0


## Calculations `3 points`

### Find how much money has been raised on Kickstarter by **successful** projects.

In [56]:
conn = sqlite3.connect("kickstarter.db")
df5 = pd.read_sql_query("select SUM(usd_pledged) from project WHERE state like 'successful'", conn)
conn.close()
df5

,SUM(usd_pledged)
0,"3,643,252,201.72"


### Find how much money has been raised on Kickstarter by "successful" projects in **the United States**.

The `state` can be successful, failed, live, etc.

In [55]:
conn = sqlite3.connect("kickstarter.db")
df6 = pd.read_sql_query("select SUM(usd_pledged) from project WHERE state like 'successful' AND country like 'US'", conn)
conn.close()
df6

,SUM(usd_pledged)
0,"2,961,700,473.92"


## Aggregating `3 points`

## How many projects were successful/unsuccessful/etc?

**Use your SQL query to calculate the answer.** Your result should look something like this:

| |...|...|
|---|---|---|
|0|canceled|36659|
|1|failed|211783|
|2|live|3163|
|3|purged|202|
|4|successful|155130|
|5|suspended|1700|

In [62]:
conn = sqlite3.connect("kickstarter.db")
df7 = pd.read_sql_query("select state, COUNT(state) from project Group by state", conn)
conn.close()
df7

,state,COUNT(state)
0,canceled,36659
1,failed,211783
2,live,3163
3,purged,202
4,successful,155130
5,suspended,1700


### How much money has been pledged to campaigns in each country?

In [89]:
conn = sqlite3.connect("kickstarter.db")
df8 = pd.read_sql_query("select country, SUM(usd_pledged) from project Group by country", conn)
conn.close()
df8

,country,SUM(usd_pledged)
0,AT,"14,451,653.89"
1,AU,"54,965,224.87"
2,BE,"5,963,505.71"
3,CA,"112,432,907.01"
4,CH,"19,435,417.64"
5,DE,"54,424,475.45"
6,DK,"11,033,924.53"
7,ES,"24,281,129.46"
8,FR,"48,536,026.19"
9,GB,"298,732,943.23"


## Conversion to CSV `2 points`

### Save the result of your last query - the one about amount raised - to a CSV

In [90]:
df8.to_csv('kickstarter_campaigns_by_country.csv')

### Visualize the results using DataWrapper, highlighting the country of your choice

Link in the cell below.

https://www.datawrapper.de/_/n2orc/

## Joins

### Show all the columns in the `category` table

In [79]:
conn = sqlite3.connect("kickstarter.db")
df9 = pd.read_sql_query("pragma table_info('category');", conn, index_col='cid')
conn.close()
df9

,name,type,notnull,dflt_value,pk
cid,,,,,
0,id,int,1,None,1
1,name,varchar(190),0,None,0
2,parent_id,int,0,None,0
3,position,int,0,None,0
4,color,int,0,None,0
5,slug,varchar(190),0,None,0
6,urls_web_discover,text,0,None,0


### Show all the columns in the `project` table

In [80]:
conn = sqlite3.connect("kickstarter.db")
df10 = pd.read_sql_query("pragma table_info('project');", conn, index_col='cid')
conn.close()
df10

,name,type,notnull,dflt_value,pk
cid,,,,,
0,id,int,1,None,1
1,state,varchar(20),0,None,0
2,url_project,varchar(190),0,None,0
3,url_project_short,varchar(190),0,None,0
4,name,varchar(190),0,None,0
5,country,varchar(20),0,None,0
6,creator_id,int,0,None,0
7,location_id,int,0,None,0
8,category_id,int,0,None,0


### Using SQL, request the first 10 projects and merge in the category of each project `3 points`

In [88]:
conn = sqlite3.connect("kickstarter.db")
df11 = pd.read_sql_query("SELECT project.name as Project, category.name as Category from project LEFT JOIN category ON category.id = project.category_id LIMIT 10", conn)
conn.close()
df11

,Project,Category
0,"ViperSharp, The Best Precision Knife Sharpener...",Product Design
1,"Lights, Coma, Action!",Nonfiction
2,Sentio Golf Putters: Feel IS the difference,Product Design
3,Brainade,Web
4,Horizon Anthology,Anthologies
5,The Alphabet Story Children's Book,Children's Books
6,Torchlight Lullaby: A Fantasy Adventure Graphi...,Graphic Novels
7,"The Gears LP, from scratch... With You.",Indie Rock
8,Polynesian Adventure - the board game,Tabletop Games
9,Masquerading Masks,Art


### Building on your last query, calculate the number of projects in each category `2 points`

In [99]:
conn = sqlite3.connect("kickstarter.db")
df12 = pd.read_sql_query("SELECT category.name as Category, count(category.name) as Number_of_Projects from project LEFT JOIN category ON category.id = project.category_id group by category.name"  , conn)
conn.close()
df12

,Category,Number_of_Projects
0,3D Printing,781
1,Academic,1062
2,Accessories,4542
3,Action,837
4,Animals,281
...,...,...
154,Woodworking,1311
155,Workshops,176
156,World Music,2265
157,Young Adult,939


### Using pandas, sort the result to get the top 20 project categories `1 point`

In [100]:
df12.sort_values("Number_of_Projects", ascending = False).head(20)

,Category,Number_of_Projects
113,Product Design,24270
90,Music,17055
136,Tabletop Games,16702
39,Documentary,15961
129,Shorts,12572
148,Video Games,12492
58,Food,11054
55,Film & Video,10523
54,Fiction,9510
10,Art,8725
